# Test de AsyncPgDbToolkit

Este notebook prueba qué métodos tiene disponible `AsyncPgDbToolkit` y cómo insertar registros correctamente.


In [1]:
import asyncio
import os
from dotenv import load_dotenv
from pgdbtoolkit import AsyncPgDbToolkit

# Cargar variables de entorno
load_dotenv()

# Configuración de la base de datos
db_config = {
    "host": os.getenv("DB_HOST", "localhost"),
    "port": int(os.getenv("DB_PORT", 5432)),
    "user": os.getenv("DB_USER", "postgres"),
    "password": os.getenv("DB_PASSWORD", ""),
    "dbname": os.getenv("DB_DATABASE", "plantcare_db")
}

print("🔍 Configuración de BD:")
print(f"   Host: {db_config['host']}")
print(f"   Port: {db_config['port']}")
print(f"   User: {db_config['user']}")
print(f"   Database: {db_config['dbname']}")


🔍 Configuración de BD:
   Host: localhost
   Port: 5432
   User: postgres
   Database: PlantCare


In [ ]:
# Crear instancia de AsyncPgDbToolkit
db = AsyncPgDbToolkit(db_config=db_config)

print("✅ Instancia de AsyncPgDbToolkit creada")
print(f"\n📋 Tipo: {type(db)}")
print(f"📋 Clase: {db.__class__.__name__}")


In [ ]:
# Listar TODOS los métodos disponibles
print("🔍 MÉTODOS DISPONIBLES EN AsyncPgDbToolkit:\n")
print("=" * 60)

methods = [method for method in dir(db) if not method.startswith('_')]
for i, method in enumerate(sorted(methods), 1):
    print(f"{i:3d}. {method}")

print(f"\n📊 Total: {len(methods)} métodos")


In [ ]:
# Verificar métodos específicos que necesitamos
needed_methods = [
    'insert_records',
    'batch_operation',
    'bulk_insert_with_copy',
    'fetch_records',
    'execute_query',
    'execute_raw_sql',
    'update_records',
    'delete_records'
]

print("🔍 VERIFICACIÓN DE MÉTODOS ESPECÍFICOS:\n")
print("=" * 60)

for method in needed_methods:
    has_method = hasattr(db, method)
    status = "✅" if has_method else "❌"
    print(f"{status} {method:25s} -> {has_method}")
    
    if has_method:
        # Obtener información del método
        method_obj = getattr(db, method)
        if callable(method_obj):
            import inspect
            try:
                sig = inspect.signature(method_obj)
                print(f"   📝 Firma: {method}{sig}")
            except:
                print(f"   📝 Es callable pero no se puede inspeccionar")
        print()


In [ ]:
# Probar conexión
async def test_connection():
    try:
        tables = await db.get_tables()
        print(f"✅ Conexión exitosa")
        print(f"📊 Tablas encontradas: {len(tables) if tables is not None else 0}")
        if tables is not None and len(tables) > 0:
            print(f"   Primeras 5 tablas: {tables[:5]}")
        return True
    except Exception as e:
        print(f"❌ Error de conexión: {e}")
        return False

result = await test_connection()


In [ ]:
# PROBAR INSERTAR UN REGISTRO - Diferentes métodos
async def test_insert_methods():
    test_data = {
        "first_name": "Test",
        "last_name": "User",
        "email": "test_insert@example.com",
        "phone": "+56912345678",
        "region": "Test Region",
        "vineyard_name": "Test Vineyard",
        "hectares": 10.0,
        "grape_type": "Test Grape",
        "password_hash": "test_hash",
        "role_id": 1,
        "active": True
    }
    
    print("🧪 PROBANDO MÉTODOS DE INSERCIÓN:\n")
    print("=" * 60)
    
    # Método 1: insert_records
    if hasattr(db, 'insert_records'):
        try:
            print("\n1️⃣ Probando insert_records...")
            result = await db.insert_records("users", test_data)
            print(f"   ✅ insert_records funcionó!")
            print(f"   📊 Resultado: {result}")
            print(f"   📊 Tipo: {type(result)}")
            
            # Limpiar
            await db.execute_query("DELETE FROM users WHERE email = %s", (test_data['email'],))
            print(f"   🧹 Registro de prueba eliminado")
        except Exception as e:
            print(f"   ❌ Error: {e}")
    else:
        print("\n1️⃣ insert_records NO existe")
    
    # Método 2: batch_operation
    if hasattr(db, 'batch_operation'):
        try:
            print("\n2️⃣ Probando batch_operation...")
            result = await db.batch_operation("insert", "users", [test_data], batch_size=1)
            print(f"   ✅ batch_operation funcionó!")
            print(f"   📊 Resultado: {result}")
            print(f"   📊 Tipo: {type(result)}")
            
            # Limpiar
            await db.execute_query("DELETE FROM users WHERE email = %s", (test_data['email'],))
            print(f"   🧹 Registro de prueba eliminado")
        except Exception as e:
            print(f"   ❌ Error: {e}")
    else:
        print("\n2️⃣ batch_operation NO existe")
    
    # Método 3: execute_query con INSERT directo (SIEMPRE DISPONIBLE)
    try:
        print("\n3️⃣ Probando execute_query con INSERT directo...")
        columns = ", ".join(test_data.keys())
        placeholders = ", ".join(["%s"] * len(test_data))
        values = list(test_data.values())
        
        query = f"INSERT INTO users ({columns}) VALUES ({placeholders}) RETURNING id"
        result = await db.execute_query(query, tuple(values))
        print(f"   ✅ execute_query con INSERT funcionó!")
        print(f"   📊 Resultado: {result}")
        print(f"   📊 Tipo: {type(result)}")
        if result is not None and not result.empty:
            print(f"   📊 ID insertado: {result.iloc[0]['id']}")
        
        # Limpiar
        await db.execute_query("DELETE FROM users WHERE email = %s", (test_data['email'],))
        print(f"   🧹 Registro de prueba eliminado")
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        print(f"   📋 Traceback: {traceback.format_exc()}")

await test_insert_methods()


In [ ]:
# PROBAR FETCH_RECORDS
async def test_fetch_methods():
    print("🧪 PROBANDO MÉTODOS DE CONSULTA:\n")
    print("=" * 60)
    
    # Método 1: fetch_records con conditions
    if hasattr(db, 'fetch_records'):
        try:
            print("\n1️⃣ Probando fetch_records con conditions...")
            result = await db.fetch_records(
                "users",
                conditions={"active": True},
                limit=5
            )
            print(f"   ✅ fetch_records funcionó!")
            print(f"   📊 Tipo: {type(result)}")
            if result is not None:
                print(f"   📊 Filas: {len(result) if hasattr(result, '__len__') else 'N/A'}")
                if hasattr(result, 'head'):
                    print(f"   📊 Primeras filas:\n{result.head()}")
        except Exception as e:
            print(f"   ❌ Error: {e}")
            import traceback
            print(f"   📋 Traceback: {traceback.format_exc()}")
    else:
        print("\n1️⃣ fetch_records NO existe")
    
    # Método 2: execute_query (SIEMPRE DISPONIBLE)
    try:
        print("\n2️⃣ Probando execute_query...")
        result = await db.execute_query(
            "SELECT * FROM users WHERE active = %s LIMIT 5",
            (True,)
        )
        print(f"   ✅ execute_query funcionó!")
        print(f"   📊 Tipo: {type(result)}")
        if result is not None:
            print(f"   📊 Filas: {len(result) if hasattr(result, '__len__') else 'N/A'}")
            if hasattr(result, 'head'):
                print(f"   📊 Primeras filas:\n{result.head()}")
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        print(f"   📋 Traceback: {traceback.format_exc()}")

await test_fetch_methods()


## 📋 RESUMEN Y RECOMENDACIONES

### ✅ Métodos que funcionan:
- **execute_query**: Para INSERT, UPDATE, DELETE, SELECT (SIEMPRE DISPONIBLE)
- **fetch_records**: Para consultas con condiciones (si existe)

### ❌ Métodos que NO existen en AsyncPgDbToolkit:
- **insert_records**: Solo en versión sincrónica (PgDbToolkit)
- **batch_operation**: Puede no estar disponible en esta versión

### 💡 SOLUCIÓN RECOMENDADA:
Usar `execute_query` con INSERT directo para `AsyncPgDbToolkit`

**Ejemplo:**
```python
columns = ", ".join(data.keys())
placeholders = ", ".join(["%s"] * len(data))
query = f"INSERT INTO table ({columns}) VALUES ({placeholders}) RETURNING id"
result = await db.execute_query(query, tuple(data.values()))
```
